# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1
    

# Example usage:
limit = 5
generator = square_root_generator(limit)


for sqrt_value in generator:
    print(sqrt_value)



1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [4]:
# Question 1: What is the sum of the outputs of the generator for limit = 5
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1
    

# Example usage:
limit = 5
generator = square_root_generator(limit)


sum_sqrt = 0
for sqrt_value in generator:
    sum_sqrt += sqrt_value

print(f"sum of the outputs of the generator for limit = 5: {sum_sqrt}")
    


sum of the outputs of the generator for limit = 5: 8.382332347441762


In [52]:
#Question 2: What is the 13th number yielded

limit = 13
generator = square_root_generator(limit)

n = 0
for sqrt_value in generator:
    n += 1
    if n == 13: 
        print(f"the 13th number yielded is: {sqrt_value}")
    else:
        continue

the 13th number yielded is: 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [5]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [6]:
import dlt
pipeline = dlt.pipeline(pipeline_name="people_data_pipeline",
						destination='duckdb', 
						dataset_name='people_dset')

In [10]:
info = pipeline.run(people_1, 
                    table_name="people", 
                    write_disposition="replace")
print(info)

Pipeline people_data_pipeline load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///D:\1_projects\data_engineer_zoomcamp\5_dlt\people_data_pipeline.duckdb location to store data
Load package 1708240524.2978008 is LOADED and contains no failed jobs


In [12]:
# append table people_2
#import dlt
pipeline = dlt.pipeline(pipeline_name="people_data_pipeline",
						destination='duckdb', 
						dataset_name='people')

info2 = pipeline.run(people_2, 
                    table_name="people", 
                    write_disposition="append")
print(info2)

Pipeline people_data_pipeline load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///D:\1_projects\data_engineer_zoomcamp\5_dlt\people_data_pipeline.duckdb location to store data
Load package 1708240705.215957 is LOADED and contains no failed jobs


In [15]:
# show outcome

import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

people = conn.sql("SELECT * FROM people").df()
display(people)

# Question 3. Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people

sum_age = conn.sql("SELECT SUM(age) from people").df()
display(sum_age)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,occupation,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,None,1708240524.2978008,vBMI0P1g5uYq8w
1,2,Person_2,27,City_A,None,1708240524.2978008,GMBxcmmNnLfCNA
2,3,Person_3,28,City_A,None,1708240524.2978008,ZBuiN6thGNUfWw
3,4,Person_4,29,City_A,None,1708240524.2978008,CO1b7UlEYhqDdw
4,5,Person_5,30,City_A,None,1708240524.2978008,D4siRRhk15Srpw
5,3,Person_3,33,City_B,Job_3,1708240705.215957,WhromIhKGQM/sg
6,4,Person_4,34,City_B,Job_4,1708240705.215957,KBVvt0fudFibmg
7,5,Person_5,35,City_B,Job_5,1708240705.215957,9wOrJwE31lDx3w
8,6,Person_6,36,City_B,Job_6,1708240705.215957,rtZz/bxO6/GQDA
9,7,Person_7,37,City_B,Job_7,1708240705.215957,EKM0y3/miT8vmQ


,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [22]:
# merge people_1 and people 2 to people_merge table

# write people_1:
info3 = pipeline.run(people_1, 
                    table_name="people_merge", 
                    write_disposition="replace")
print(info3)



Pipeline people_data_pipeline load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///D:\1_projects\data_engineer_zoomcamp\5_dlt\people_data_pipeline.duckdb location to store data
Load package 1708246937.477226 is LOADED and contains no failed jobs


In [23]:
# merge people_2 to people_1 using id column
info4 = pipeline.run(people_2, 
                    table_name="people_merge", 
                    write_disposition="merge",
                    primary_key="id") 
print(info4)

Pipeline people_data_pipeline load step completed in 0.30 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///D:\1_projects\data_engineer_zoomcamp\5_dlt\people_data_pipeline.duckdb location to store data
Load package 1708246939.79661 is LOADED and contains no failed jobs


In [24]:
# show outcome

import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_merge table below:")

people = conn.sql("SELECT * FROM people_merge").df()
display(people)

# Question 3. Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people

sum_age = conn.sql("SELECT SUM(age) from people_merge").df()
display(sum_age)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_merge        │
└─────────────────────┘




 people_merge table below:


,id,name,age,city,occupation,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,None,1708246937.477226,fqBcgwODjXKnHg
1,2,Person_2,27,City_A,None,1708246937.477226,jFPVjP84ZXsZCQ
2,5,Person_5,35,City_B,Job_5,1708246939.79661,bNXdyYX/epsPOg
3,7,Person_7,37,City_B,Job_7,1708246939.79661,YDZyFogMWrSqnw
4,8,Person_8,38,City_B,Job_8,1708246939.79661,ZV/kLIdPKXJMkg
5,3,Person_3,33,City_B,Job_3,1708246939.79661,56HdiPZzRZ2rrQ
6,6,Person_6,36,City_B,Job_6,1708246939.79661,YNMX4clNIZeRgg
7,4,Person_4,34,City_B,Job_4,1708246939.79661,K+pXcEB0p13BZA


,sum(age)
0,266.0


# Solution: First make sure that the following modules are installed:

In [1]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

UsageError: Line magic function `%%capture` not found.


In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX